In [1]:
# Want to load in all data for a given datatype, i.e. human, zebra, mouse
# then catalogue relevant info for each instance:
#
# fname inst# n m n_red m_red k_gt k_opt cutset impr_bd t_w t_path

froot = 'zebra'
timeoutexceed = 800
which_alg = "toboggan"

In [2]:
import collections
inputfile = "all-" + froot + ".txt"

def make_tables(inputfile):
    datamatrix = []
    datadict = collections.defaultdict(int)
    idx = 0
    with open(inputfile, 'r') as datafile:
        for line in datafile:
            temp_line = line.strip().split()
            key = temp_line[0] + ' ' + temp_line[1]
            row = temp_line[2::]
            if datadict[key] == 0:
                datamatrix.append(row)
                datadict[key] = idx
                idx +=1
            else:  # check which row has longer timeout
                newtime = float(row[8])
                oldtime = float(datamatrix[datadict[key]][8])
                if newtime > oldtime:
                    datamatrix[datadict[key]] = row
    return datadict, datamatrix


In [3]:
datadict, datamatrix = make_tables(inputfile)

In [8]:
num_trivial = 0
num_timedout = 0
time_totals = []
total_num = len(datamatrix)
for key, val in datadict.items():
    row = datamatrix[val]
    if row[2] == '1':  # trivial instance
        num_trivial += 1
        continue
    elif row[5] == 'None':  # timedout instance
        num_timedout += 1
        continue
    else:
        time_total = float(row[8]) + float(row[9])
        if time_total > 590:
            #print("time value exceeds timeout")
            print(key)
            #print(row)
            continue
        if time_total == 0.0:
            print("Time of 0.0, something went wrong")
            print(froot + " " + key)
            print(row)
        time_totals.append(time_total)

zebra 23.graph 13037
23.graph 13037


In [5]:
max(time_totals)

591.556965828

In [6]:
import matplotlib.pyplot as plt
import numpy as np

plt.clf()
#logbins=np.logspace(-4, 3, 7)
logbins = [10**-4, 10**-3, 10**-2, 10**-1, 1, 10, 100, 1000]
n, bins, patchs = plt.hist(time_totals, bins=logbins, log=True)
plt.title("Runtimes: {}, {}".format(which_alg, froot))
plt.xlabel("time")
plt.ylabel("Frequency")
plt.gca().set_xscale("log")

#plt.show()
plt.savefig( ''.join(['runtime-toboggan-', froot, '.eps']), format='eps', dpi=300)
plt.savefig( ''.join(['runtime-toboggan-', froot, '.png']), format='png', dpi=300)
plt.close()